<a href="https://colab.research.google.com/github/chauhanneha16/UniversityRecommendationApp/blob/main/University_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [21]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle
import streamlit as st

# Create the sample dataset
data = {
    "University Name": ["Oxford", "MIT", "Harvard", "Cambridge", "Tokyo University"],
    "Science Marks": [85, 90, 88, 87, 86],
    "Maths Marks": [90, 95, 92, 91, 93],
    "History Marks": [80, 75, 78, 79, 77],
    "English Marks": [85, 80, 86, 84, 82],
    "GRE Marks": [320, 330, 325, 315, 310],
    "Course": ["Physics", "Computer Science", "History", "Engineering", "Chemistry"]
}

df = pd.DataFrame(data)

# Preprocess the data
# Normalize the marks
scaler = StandardScaler()
marks_columns = ["Science Marks", "Maths Marks", "History Marks", "English Marks", "GRE Marks"]
df[marks_columns] = scaler.fit_transform(df[marks_columns])

# Encode the university names and courses
label_encoder_uni = LabelEncoder()
df["University Name"] = label_encoder_uni.fit_transform(df["University Name"])

label_encoder_course = LabelEncoder()
df["Course"] = label_encoder_course.fit_transform(df["Course"])

# Save the scaler and label encoders
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('label_encoder_course.pkl', 'wb') as f:
    pickle.dump(label_encoder_course, f)
with open('label_encoder_uni.pkl', 'wb') as f:
    pickle.dump(label_encoder_uni, f)

# Split the data into input features and target labels
X = df[marks_columns]
y = df["University Name"]

# Build the model
model = Sequential([
    Dense(64, input_dim=X.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(df["University Name"].unique()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50, batch_size=2)

# Save the model
model.save('university_recommendation_model.h5')

# Function to predict the university, recommended course, and advice based on input marks
def predict_university_and_advice(science_marks, maths_marks, history_marks, english_marks, gre_marks):
    input_data = np.array([[science_marks, maths_marks, history_marks, english_marks, gre_marks]])
    input_data = scaler.transform(input_data)
    prediction = model.predict(input_data)
    predicted_uni_index = np.argmax(prediction)
    predicted_uni = label_encoder_uni.inverse_transform([predicted_uni_index])[0]

    # Determine the recommended course based on the highest marks
    marks_dict = {
        "Science": science_marks,
        "Maths": maths_marks,
        "History": history_marks,
        "English": english_marks,

    }
    highest_mark_subject = max(marks_dict, key=marks_dict.get)

    if highest_mark_subject == "Science":
        recommended_course = "MBBS"
    elif highest_mark_subject == "Maths":
        recommended_course = "Engineering"
    elif highest_mark_subject == "History":
        recommended_course = "History"
    elif highest_mark_subject == "English":
        recommended_course = "Literature"


    # Personalized advice
    advice = []

    if science_marks < 50:
        advice.append("Focus on improving your Science marks.")
    if maths_marks < 50:
        advice.append("Focus on improving your Maths marks.")
    if history_marks < 50:
        advice.append("Focus on improving your History marks.")
    if english_marks < 50:
        advice.append("Focus on improving your English marks.")
    if gre_marks < 200:
        advice.append("Consider retaking the GRE to improve your score.")

    if recommended_course == "Engineering":
        advice.append("Enhance your programming skills by taking online courses.")
    elif recommended_course == "Computer Science":
        advice.append("Work on projects and internships related to software development.")
    elif recommended_course == "Physics":
        advice.append("Participate in research projects and science fairs.")
    elif recommended_course == "History":
        advice.append("Read extensively and engage in historical research projects.")
    elif recommended_course == "Chemistry":
        advice.append("Gain hands-on experience in laboratories and participate in chemistry competitions.")
    elif recommended_course == "MBBS":
        advice.append("Gain practical experience by volunteering at clinics or hospitals.")
    elif recommended_course == "Literature":
        advice.append("Engage in extensive reading and writing practice.")

    return predicted_uni, recommended_course, advice

# Example prediction
science_marks = 30
maths_marks = 90
history_marks = 50
english_marks = 100
gre_marks = 100


predicted_university, recommended_course, advice = predict_university_and_advice(science_marks, maths_marks, history_marks, english_marks, gre_marks)
print(f"The recommended university is: {predicted_university}")
print(f"The recommended course is: {recommended_course}")
print(f"Personal advice: {', '.join(advice)}")

# Streamlit app layout
st.title('University Recommendation System')

science_marks = st.number_input('Science Marks', min_value=0.0, max_value=100.0)
maths_marks = st.number_input('Maths Marks', min_value=0.0, max_value=100.0)
history_marks = st.number_input('History Marks', min_value=0.0, max_value=100.0)
english_marks = st.number_input('English Marks', min_value=0.0, max_value=100.0)
gre_marks = st.number_input('GRE Marks', min_value=0.0, max_value=340.0)

if st.button('Submit'):
    university, course, advice = predict_university_and_advice(science_marks, maths_marks, history_marks, english_marks, gre_marks)
    st.write(f"Recommended University: {university}")
    st.write(f"Recommended Course: {course}")
    st.write("Personal Advice:")
    for item in advice:
        st.write(f"- {item}")



Epoch 1/50
3/3 [==============================] - 1s 6ms/step - loss: 1.7083 - accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 6ms/step - loss: 1.5942 - accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 7ms/step - loss: 1.4951 - accuracy: 0.4000
Epoch 4/50
3/3 [==============================] - 0s 5ms/step - loss: 1.4025 - accuracy: 0.6000
Epoch 5/50
3/3 [==============================] - 0s 6ms/step - loss: 1.3269 - accuracy: 0.6000
Epoch 6/50
3/3 [==============================] - 0s 5ms/step - loss: 1.2652 - accuracy: 0.8000
Epoch 7/50
3/3 [==============================] - 0s 5ms/step - loss: 1.1953 - accuracy: 0.8000
Epoch 8/50
3/3 [==============================] - 0s 5ms/step - loss: 1.1351 - accuracy: 1.0000
Epoch 9/50
3/3 [==============================] - 0s 5ms/step - loss: 1.0851 - accuracy: 1.0000
Epoch 10/50
3/3 [==============================] - 0s 5ms/step - loss: 1.0312 - accuracy: 1.0000
Epoch 11/50
3/3 [==============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
